In [9]:
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt

from nltk import word_tokenize 
from nltk.util import ngrams
from nltk.corpus import stopwords
from tqdm.notebook import tqdm_notebook as tqdm

from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# read in the HT data file
data_file = "../Data/ComplexDataLab_Canadian_OctoberBatch.csv"
data = pd.read_csv(data_file, index_col=0)
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,PostingDate,Url,l_Location,u_Age,u_Description,u_Title,u_Location,e_PhoneNumbers,u_PhoneNumbers,e_Emails
0,Thu 14 Apr 2016 11:40 AM,https://independent.com.calgary.listcrawler.co...,calgary,31,Take the time to enjoy your night with me.. 40...,Sinfully sweet,"Calgary, Downtown, Mins from Downtown, Northea...",['4039715391'],['4039715391'],[]
1,Tue 04 Jul 2017 08:30 AM,https://yolo.com.calgary.listcrawler.com/post/...,calgary,28,Maria Vega Latina for Real!!!Plz read my rate...,Tired fake AddsFake Expectations36 DDD Real 10...,"Calgary, Downtown, DOWNTOWN CORE INCALL and OU...",['4389267478'],['4389267478'],[]
2,Mon 03 Jul 2017 07:04 AM,https://milfy.com.toronto.listcrawler.com/post...,toronto,34,Sexual Deviant LOVES 2ROCK IT NAUGHTY! My sexu...,New Halton Girl! Outcall MIND BL0-WING DEVIANT...,"Burlington, Hamilton, Oakville milton, Oakvill...",['2897801427'],['2897801427'],[]
3,Thu 29 Sep 2016 12:06 AM,https://max80.com.vancouver.listcrawler.com/po...,vancouver,25,778 8four6 4234 Hi! I'm Tiffany baby! ;) LETS ...,Early Morning Drop In! Rates $40-$100 TEXT NOW!,"City of Vancouver, MARINE &amp; CAMBIE, Vancouver",[],['7788464234'],[]
4,Tue 03 May 2016 06:16 PM,https://milfy.com.barrie.listcrawler.com/post/...,barrie,33,HI GUYS ! I CANT WAIT TO MEET YOU ! I AM 33 5...,"BARBIE aka (Cheyenne) 33 fetishes welcome, soa...","Barrie, Barrie and surrounding areas",['7053314617'],['7053314617'],[]


In [3]:
# size of the data
data.shape

(932888, 10)

In [4]:
data = data.drop_duplicates()
data.shape

(932888, 10)

In [6]:
# randomly sample some data points 
sampled_data = data.sample(n=5000)

In [7]:
sampled_data.head()

,PostingDate,Url,l_Location,u_Age,u_Description,u_Title,u_Location,e_PhoneNumbers,u_PhoneNumbers,e_Emails
642087,Fri 18 Aug 2017 08:33 PM,https://candy.com.montreal.listcrawler.com/pos...,montreal,21,"Hi boys :) Lexie21 years old, 5'6 125lbs Blon...",Lexie --- 438-800-9850100% Independant,"City of Montreal, Montreal",['4388009850'],['4388009850'],[]
575134,Fri 02 Jun 2017 12:25 AM,https://candy.com.brantford.listcrawler.com/po...,brantford,32,Brittney JUICY VOLUPTUOUS % all CANADIAN CUTIE...,Coming to Town tomorrow BOOK NOW SEXY CURVY ...,"Brantford-Woodstock, Woodstock",['2898316607'],['2898316607'],[]
570482,Wed 16 Mar 2016 03:09 PM,https://candy.com.toronto.listcrawler.com/post...,toronto,21,your dream girl is here! I'm Ms StacyThis is a...,VB B P #1M$TD 647-713-9111,"427/Dundas, Brampton, City of Toronto, Mississ...",['6477139111'],['6477139111'],[]
801067,Sat 16 Feb 2019 03:33 AM,https://blackdynomite.com.toronto.listcrawler....,toronto,21,hi gentlemen My name is destiny ’m new in INT...,Come taste the sweetest caramel chocolate,Mississauga,['4167192068'],['6477192068'],[]
33314,Wed 14 Jun 2017 05:27 PM,https://aypapi.com.vancouver.listcrawler.com/p...,vancouver,22,Hey gentlemen!< A Lets have some fun .. I'm a ...,the perfect treat you haven't had the b...,"Airport, Richmond, Vancouver",['6477999487'],['6477999487'],[]


In [10]:
data['label'] = None
data['group'] = None
grp_id = 1
for grp in tqdm(data.groupby('u_PhoneNumbers')):
    ids = grp[1].index.values
    if len(ids) <= 1:
        data['label'].iloc[ids] = 0
    else:
        data['label'].iloc[ids] = 1
    data['group'].iloc[ids] = grp_id
    grp_id += 1

In [11]:
data.head()

,PostingDate,Url,l_Location,u_Age,u_Description,u_Title,u_Location,e_PhoneNumbers,u_PhoneNumbers,e_Emails,label,group
0,Thu 14 Apr 2016 11:40 AM,https://independent.com.calgary.listcrawler.co...,calgary,31,Take the time to enjoy your night with me.. 40...,Sinfully sweet,"Calgary, Downtown, Mins from Downtown, Northea...",['4039715391'],['4039715391'],[],1,23710
1,Tue 04 Jul 2017 08:30 AM,https://yolo.com.calgary.listcrawler.com/post/...,calgary,28,Maria Vega Latina for Real!!!Plz read my rate...,Tired fake AddsFake Expectations36 DDD Real 10...,"Calgary, Downtown, DOWNTOWN CORE INCALL and OU...",['4389267478'],['4389267478'],[],1,31764
2,Mon 03 Jul 2017 07:04 AM,https://milfy.com.toronto.listcrawler.com/post...,toronto,34,Sexual Deviant LOVES 2ROCK IT NAUGHTY! My sexu...,New Halton Girl! Outcall MIND BL0-WING DEVIANT...,"Burlington, Hamilton, Oakville milton, Oakvill...",['2897801427'],['2897801427'],[],1,16253
3,Thu 29 Sep 2016 12:06 AM,https://max80.com.vancouver.listcrawler.com/po...,vancouver,25,778 8four6 4234 Hi! I'm Tiffany baby! ;) LETS ...,Early Morning Drop In! Rates $40-$100 TEXT NOW!,"City of Vancouver, MARINE &amp; CAMBIE, Vancouver",[],['7788464234'],[],1,80398
4,Tue 03 May 2016 06:16 PM,https://milfy.com.barrie.listcrawler.com/post/...,barrie,33,HI GUYS ! I CANT WAIT TO MEET YOU ! I AM 33 5...,"BARBIE aka (Cheyenne) 33 fetishes welcome, soa...","Barrie, Barrie and surrounding areas",['7053314617'],['7053314617'],[],1,75746


In [12]:
data.to_csv("data_with_labels.csv",index=None)

In [68]:
sampled_data = pd.concat([data[data.label==1].sample(n=500), data[data.label==0].sample(n=500)])

In [69]:
sampled_data = sampled_data.sample(frac=1)

In [70]:
def get_ngrams(text):
    tokens = word_tokenize(text)
    tokensProcessed = [token.lower() for token in tokens if token not in stopwords.words('english')]
    bigrams = list(ngrams(tokensProcessed, 2))
    unigrams = list(ngrams(tokensProcessed, 1))
    return unigrams, bigrams

In [71]:
def get_loc_features(text):
    vectorizer = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1,1))
    features = vectorizer.fit_transform(text)
    return features.toarray(), vectorizer.get_feature_names()

In [72]:
def get_features(text):
    vectorizer_uni = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(1,1))
    features_uni = vectorizer_uni.fit_transform(text)
    vectorizer_bi = CountVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words('english'), ngram_range=(2,2))
    features_bi = vectorizer_bi.fit_transform(text)
    return features_uni.toarray(), vectorizer_uni.get_feature_names(), features_bi.toarray(), vectorizer_bi.get_feature_names()

In [105]:
desc_features_uni, desc_feature_names_uni, desc_features_bi, desc_feature_names_bi = get_features(sampled_data.u_Description)
title_features_uni, title_feature_names_uni, title_features_bi, title_feature_names_bi = get_features(sampled_data.u_Title)

In [106]:
loc_features, loc_feature_names = get_loc_features(sampled_data.u_Location)

In [75]:
title_features_uni.shape

(1000, 3134)

In [107]:
feature_vectors = np.concatenate((desc_features_uni, desc_features_bi, title_features_uni, title_features_bi, title_features_bi), axis=1)

In [108]:
feature_vectors.shape

(1000, 55607)

In [109]:
sampled_data_labels = sampled_data.label.values

In [110]:
pkl.dump(sampled_data_labels, open("sample_data/sampled_data_labels.pkl","wb"))
pkl.dump(feature_vectors, open("sample_data/sampled_data_features.pkl","wb"), protocol=4)

In [96]:
feature_names = np.append(np.append(np.append(np.append(desc_feature_names_uni, desc_feature_names_bi), title_feature_names_uni), title_feature_names_bi), loc_feature_names)

In [113]:
pkl.dump(feature_names, open("sample_data/sampled_data_feature_names.pkl","wb"), protocol=4)

In [112]:
feature_names = list(set(np.concatenate([desc_feature_names_uni, desc_feature_names_bi, title_feature_names_uni, title_feature_names_bi, loc_feature_names])))

In [114]:
# mapping feature words to ids
word_to_id_map = {}
counter = 1
for word in tqdm(set(feature_names)):
    word_to_id_map[word] = -counter
    counter += 1

In [115]:
desc_feature_names_uni_values = []
for word in set(desc_feature_names_uni):
    desc_feature_names_uni_values.append(word_to_id_map[word])

desc_feature_names_bi_values = []
for word in set(desc_feature_names_bi):
    desc_feature_names_bi_values.append(word_to_id_map[word])
    
title_feature_names_uni_values = []
for word in set(title_feature_names_uni):
    title_feature_names_uni_values.append(word_to_id_map[word])
    
title_feature_names_bi_values = []
for word in set(title_feature_names_bi):
    title_feature_names_bi_values.append(word_to_id_map[word])
    
loc_feature_names_values = []
for word in set(loc_feature_names):
    loc_feature_names_values.append(word_to_id_map[word])

In [119]:
pkl.dump(desc_feature_names_uni_values, open("sample_data/desc_feature_names_uni.pkl","wb"))
pkl.dump(desc_feature_names_bi_values, open("sample_data/desc_feature_names_bi.pkl","wb"))
pkl.dump(title_feature_names_uni_values, open("sample_data/title_feature_names_uni.pkl","wb"))
pkl.dump(title_feature_names_bi_values, open("sample_data/title_feature_names_bi.pkl","wb"))
pkl.dump(loc_feature_names_values, open("sample_data/loc_feature_names.pkl","wb"))

In [120]:
pkl.dump(word_to_id_map, open("sample_data/feature_name_to_id_map.pkl","wb"))